In [1]:
import argparse
import sqlite3
import shutil
import time
import os

In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict

C:\Users\Elf\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Elf\anaconda3\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Users\Elf\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Пример работы с БД в Python

In [3]:
conn = sqlite3.connect('test_database') 
c = conn.cursor()

In [4]:
def close_connection():
    conn.commit()
    conn.close()

Создание БД

In [5]:
c.execute('''
          CREATE TABLE IF NOT EXISTS products
          ([product_id] INTEGER PRIMARY KEY, [product_name] TEXT)
          ''')

Заполнение БД

In [6]:
# c.execute('''
#           INSERT INTO products (product_id, product_name)

#                 VALUES
#                 (1,'Computer'),
#                 (2,'Printer'),
#                 (3,'Tablet'),
#                 (4,'Desk'),
#                 (5,'Chair')
#           ''')

SQL-запрос и вывод результатов в Python

In [7]:
c.execute('''
select * from products
'''
)

In [8]:
pd.DataFrame(c.fetchall(), columns=['product_name','price'])

,product_name,price


# ДЗ

## Задание 1 [3 балла]. Создайте схему БД "Университет".

Она должна включать в себя минимально следующие отношения:

    - студенты (минимальные атрибуты: ФИО, номер зачётки, дата рождения, группа, год поступления)
    - посещаемые курсы (минимальные атрибуты: название курса, тип курса, дата введения в программу)
    - преподаватели (минимальные атрибуты: ФИО, дата рождения, дата начала работы в нашем университете, дата начала трудовой деятельности)
Атрибуты отношений должны быть осмысленными и соответствовать тематике.

In [9]:
conn = sqlite3.connect('university_database') 
c = conn.cursor()

- студенты (минимальные атрибуты: ФИО, номер зачётки, дата рождения, группа, год поступления)


In [10]:
students_cols = ['record_book_id', 'name', 'birth_date', 'group_id', 'enrollment_year']

c.execute('''
          CREATE TABLE IF NOT EXISTS students
          ([record_book_id] INTEGER PRIMARY KEY, 
          [name] TEXT,
          [birth_date] DATE,
          [group_id] TEXT,
          [enrollment_year] INTEGER)
          ''')

'''
ALTER TABLE students 
RENAME COLUMN unit TO group_number
'''

'\nALTER TABLE students \nRENAME COLUMN unit TO group_number\n'

In [11]:
c.execute('''
select * from students
'''
)
pd.DataFrame(c.fetchall(), columns=students_cols)

,record_book_id,name,birth_date,group_id,enrollment_year


- посещаемые курсы (минимальные атрибуты: название курса, тип курса, дата введения в программу)

In [12]:
courses_cols = ['course_id', 'name', 'type', 'inclusion_date']

c.execute('''
          CREATE TABLE IF NOT EXISTS courses
          ([course_id] INTEGER PRIMARY KEY, 
          [name] TEXT,
          [type] TEXT,
          [inclusion_date] DATE)
          ''')

 Связь курсов и студентов

In [13]:
c.execute('''
          CREATE TABLE IF NOT EXISTS students_courses
          ([record_book_id] INTEGER NOT NULL,
          [course_id] INTEGER NOT NULL,
          FOREIGN KEY (record_book_id) REFERENCES students(record_book_id),
          FOREIGN KEY (course_id) REFERENCES courses(course_id))
          ''')

- преподаватели (минимальные атрибуты: ФИО, дата рождения, дата начала работы в нашем университете, дата начала трудовой деятельности)

In [14]:
professors_cols = ['professor_id', 'name', 'birth_date', 'job_started', 'profession_started']

c.execute('''
          CREATE TABLE IF NOT EXISTS professors
          ([professor_id] INTEGER PRIMARY KEY,
          [name] TEXT,
          [birth_date] DATE,
          [job_started] DATE,
          [profession_started] DATE)
          ''')

Связь преподавателей и курсов

In [15]:
c.execute('''
          CREATE TABLE IF NOT EXISTS professors_courses
          ([professor_id] INTEGER NOT NULL,
          [course_id] INTEGER NOT NULL,
          FOREIGN KEY (professor_id) REFERENCES professors(professor_id),
          FOREIGN KEY (course_id) REFERENCES courses(course_id))
          ''')

## Задание 2 [1 балл]. 
Заполните вашу схему фейковыми записями - в отношении Студенты должно быть не менее 150 строк

В остальных - не менее 20 строк.

Используем чудесные генераторы данных

In [16]:
!pip install Faker russian_names


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
students_cols

['record_book_id', 'name', 'birth_date', 'group_id', 'enrollment_year']

In [18]:
from russian_names import RussianNames
from faker import Faker


name_generator = RussianNames()
fake = Faker()

unique_ids = np.array(list(set([np.random.randint(1e5, 1e6) for _ in range(1000)])))

students_id = unique_ids[np.random.choice(len(unique_ids), size=200, replace=False)]
courses_id = unique_ids[np.random.choice(len(unique_ids), size=20, replace=False)]
professors_id = unique_ids[np.random.choice(len(unique_ids), size=30, replace=False)]

### students

In [19]:
student_data = defaultdict(list)

for i, record_book_id in enumerate(students_id):
    student_data["record_book_id"].append(record_book_id)
    student_data["name"].append(name_generator.get_person())
    
    birth_date = fake.date_of_birth(minimum_age = 17, maximum_age = 27)
    
    student_data["birth_date"].append(birth_date)
    student_data["group_id"].append(np.random.randint(1, 5))
    student_data["enrollment_year"].append(min(birth_date.year + np.random.randint(18, 27), 2022))

In [20]:
df_student_data = pd.DataFrame(student_data)

df_student_data.to_sql('students', conn, if_exists='append', index=False)

c.execute('''
select * from students
'''
)
pd.DataFrame(c.fetchall(), columns=students_cols)

,record_book_id,name,birth_date,group_id,enrollment_year
0,110830,Ева Михайловна Фомыгина,1997-05-07,3,2021
1,117554,Александра Викторовна Дорогинина,1995-08-23,4,2020
2,119213,Алла Егоровна Шерватова,2004-03-26,3,2022
3,119310,Яна Егоровна Ахметьева,2003-08-19,4,2022
4,123984,Тарас Яковлевич Новасадов,1997-12-16,1,2022
...,...,...,...,...,...
195,980112,Карина Вадимовна Жичкина,2002-05-28,1,2022
196,983076,Василий Иванович Гомегузов,2004-11-30,3,2022
197,985069,Степан Фёдорович Минцов,2001-03-07,1,2022
198,986180,Артур Иванович Кочкаев,2003-12-04,1,2022


### courses

In [21]:
courses_cols

['course_id', 'name', 'type', 'inclusion_date']

In [22]:
course_names = np.array(["Алгебра", "Физика", "Криптография", "Математический анализ", "C++", "Python", 
                         "Базы данных", "Топология", "Экономика", "Теория вероятностей", "Мат. Статистика",
                        "Машинное обучение", "Анализ данных", "Глубокое обучение", "Обработка текста",
                        "Обработка звука", "Стохастические процессы", "Теоретическая механика", 
                         "Теория случайных процессов", "Дискретная математика"])

course_type = np.array(["обязательный", "факультативный", "курс по выбору"])

In [23]:
course_data = defaultdict(list)

for i, course_id in enumerate(courses_id):
    course_data["course_id"].append(course_id)
    course_data["name"].append(course_names[i])
    course_data["type"].append(course_type[np.random.randint(0, len(course_type))])
    course_data["inclusion_date"].append(np.random.randint(2010, 2022))

In [24]:
df_course_data = pd.DataFrame(course_data)

df_course_data.to_sql('courses', conn, if_exists='append', index=False)

c.execute('''
select * from courses
'''
)
pd.DataFrame(c.fetchall(), columns=courses_cols)

,course_id,name,type,inclusion_date
0,165381,Математический анализ,факультативный,2019
1,196897,Теория вероятностей,курс по выбору,2013
2,226567,Стохастические процессы,факультативный,2011
3,240266,Базы данных,обязательный,2017
4,288311,Машинное обучение,обязательный,2012
5,337259,Теория случайных процессов,курс по выбору,2011
6,539706,Криптография,курс по выбору,2016
7,591231,Обработка звука,курс по выбору,2013
8,618490,Физика,обязательный,2011
9,641517,Дискретная математика,обязательный,2013


### students_courses

In [25]:
students_courses_data = defaultdict(list)

for i, student_id in enumerate(students_id):
    n_courses = np.random.randint(2, 20)
    for course_id in np.random.choice(len(courses_id), size=n_courses, replace=False):
        students_courses_data['record_book_id'].append(student_id)
        students_courses_data['course_id'].append(courses_id[course_id])

In [26]:
df_students_courses_data = pd.DataFrame(students_courses_data)

df_students_courses_data.to_sql('students_courses', conn, if_exists='append', index=False)

c.execute('''
select * from students_courses
'''
)
pd.DataFrame(c.fetchall(), columns=['record_book_is', 'course_id'])

,record_book_is,course_id
0,743753,165381
1,743753,618490
2,743753,751111
3,743753,226567
4,743753,687897
...,...,...
2034,624047,936573
2035,624047,240266
2036,624047,687897
2037,624047,539706


### professors

In [27]:
professors_cols

['professor_id', 'name', 'birth_date', 'job_started', 'profession_started']

In [28]:
professors_data = defaultdict(list)

for i, professor_id in enumerate(professors_id):
    professors_data["professor_id"].append(professor_id)
    professors_data["name"].append(name_generator.get_person())
    
    birth_date = fake.date_of_birth(minimum_age = 17, maximum_age = 78)
    
    professors_data["birth_date"].append(birth_date)
    profession_started = min(birth_date.year + np.random.randint(22, 32), 2022)
    professors_data["job_started"].append(min(profession_started + np.random.randint(0, 20), 2022))
    professors_data["profession_started"].append(profession_started)

In [29]:
df_professors_data = pd.DataFrame(professors_data)

df_professors_data.to_sql('professors', conn, if_exists='append', index=False)

c.execute('''
select * from professors
'''
)
pd.DataFrame(c.fetchall(), columns=professors_cols)

,professor_id,name,birth_date,job_started,profession_started
0,152905,Максим Леонидович Калитрдин,1953-02-25,1989,1980
1,168429,Григорий Кириллович Саховдинов,1982-12-07,2008,2006
2,169953,Евгений Данилович Гугучкин,1967-12-12,1993,1992
3,187697,Леонид Викторович Ширинский,1970-12-18,2020,2001
4,203025,Анна Богдановна Аллаярова,1991-11-12,2022,2018
5,224059,Денис Петрович Гуцулаев,2001-11-07,2022,2022
6,260907,Дмитрий Алексеевич Хахонин,1994-06-12,2022,2022
7,328275,Владимир Борисович Жавинский,1954-12-03,1987,1977
8,352883,Вадим Юриевич Бобылкин,1968-10-03,1999,1999
9,382784,Вячеслав Антонович Пупшев,2001-12-26,2022,2022


### professors_courses

In [30]:
professors_courses_data = defaultdict(list)

for i, professor_id in enumerate(professors_id):
    n_courses = np.random.randint(3, 7)
    for course_id in np.random.choice(len(courses_id), size=n_courses, replace=False):
        professors_courses_data['professor_id'].append(professor_id)
        professors_courses_data['course_id'].append(courses_id[course_id])

In [31]:
df_professors_courses_data = pd.DataFrame(professors_courses_data)

df_professors_courses_data.to_sql('professors_courses', conn, if_exists='append', index=False)

c.execute('''
select * from professors_courses
'''
)
pd.DataFrame(c.fetchall(), columns=['professor_id', 'course_id'])

,professor_id,course_id
0,461343,770891
1,461343,784991
2,461343,196897
3,461343,762181
4,461343,936573
...,...,...
122,638215,618490
123,168429,539706
124,168429,671014
125,168429,337259


Задание 3 [1 балл]. Выведите топ-10 курсов, которые ведут наименьшее число преподавателей. Расположите в порядке возрастания числа преподавателей.

In [32]:
c.execute('''
select c.name, prof_cnt.cnt from courses as c
join
(select course_id, count(distinct professor_id) as cnt from professors_courses
group by course_id
order by count(distinct professor_id) asc
limit 10) as prof_cnt
on c.course_id = prof_cnt.course_id
''')


pd.DataFrame(c.fetchall(), columns=['name', 'count'])

,name,count
0,Алгебра,2
1,Базы данных,3
2,Машинное обучение,3
3,Python,4
4,Криптография,5
5,Физика,5
6,Дискретная математика,5
7,Анализ данных,6
8,Экономика,6
9,Обработка текста,6


Задание 4 [2 балла]. Напишите запрос, который выведет топ-10 преподавателей, которые преподают у наибольшего числа студентов. Расположите их в порядка убывания числа студентов.

In [33]:
c.execute('''
select p.name, topprof.st_cnt from professors as p
join 
(select pc.professor_id, count(distinct sc.record_book_id) as st_cnt from professors_courses as pc
join students_courses as sc on pc.course_id = sc.course_id
group by pc.professor_id
order by count(distinct sc.record_book_id) desc
limit 10) as topprof 
on p.professor_id = topprof.professor_id
''')

pd.DataFrame(c.fetchall(), columns=['name', 'count'])

,name,count
0,Анна Богдановна Аллаярова,189
1,Катерина Валерьевна Кармашова,187
2,Клавдия Богдановна Тогреева,185
3,Софья Богдановна Петрашкина,184
4,Артём Алексиевич Щульцев,183
5,Наталия Ивановна Сенишина,183
6,Артур Виталиевич Выроцков,183
7,Вячеслав Антонович Пупшев,182
8,Валерий Виталиевич Солодилов,180
9,Клара Андреевна Егнарова,180


Задание 5 [1.5 балла]. Для каждой группы студентов, поступивших не ранее 2019 года, выведите размах между максимальной и минимальной датами рождений студентов в этой группе.

In [34]:
c.execute(
    '''
    select group_id, max(birth_date) - min(birth_date), max(birth_date), min(birth_date) from students
    where enrollment_year >= 2019
    group by group_id
    '''
)

pd.DataFrame(c.fetchall(), columns = ['group_id', 'birthday variety', 'youngest', 'eldest'])

,group_id,birthday variety,youngest,eldest
0,1,10,2005-11-14,1995-11-02
1,2,11,2006-01-30,1995-08-25
2,3,11,2006-01-10,1995-05-23
3,4,10,2005-08-20,1995-08-23


Задание 6 [1.5 балла]. Среди преподавателей, родившихся в период между самой ранней датой начала работы в нашем университете и самой ранней датой начала трудовой деятельности по всем преподавателям, определите самое популярное имя.

Примечание: данные генерируются случайно, вероятность того, что у кого-то совпадут имена очень мала

In [44]:
c.execute(
    '''
    select SUBSTR(name, 1, instr(name, ' ') - 1) AS firstname from professors
    where cast(cast(birth_date as date) as text) between 
    cast((select min(profession_started) from professors) as text)
    and
    cast((select min(job_started) from professors) as text)
    group by firstname
    order by count(distinct SUBSTR(name, 1, instr(name, ' ') - 1)) desc
    limit 1
    '''
)

pd.DataFrame(c.fetchall())

,0
0,Леонид
